Libraries

In [1]:
import warnings
warnings.filterwarnings('ignore')
import os
import glob
import librosa
import scipy.io.wavfile as wave
from scipy.fftpack import fft, ifft, fftshift
import matplotlib.pyplot as plt 
import numpy as np
from sklearn import mixture
import joblib
import pandas as pd
from sklearn.mixture import GaussianMixture
import json
from scipy.io import loadmat


Matlab Script

In [2]:
import matlab.engine
eng = matlab.engine.start_matlab()


ModuleNotFoundError: No module named 'matlab'

Variables

In [ ]:

n_mixtures = 7
max_iterations = 200
#calc_deltas=True
#sr=8000
#hop_length=int(0.005*sr)

Training Function

In [ ]:
def training(train_data_path,feat_train_path,trained_model_path):
  all_speakers=glob.glob(train_data_path+'*')
  directory=feat_train_path
  if not os.path.exists(directory):
      os.makedirs(directory)
      
  directory=trained_model_path
  if not os.path.exists(directory):
      os.makedirs(directory)

      
  for itr1 in range(0,len(all_speakers)):
      mp3 = glob.glob(all_speakers[itr1]+'/*.mp3')
      for itr3 in range(0,len(mp3)):
        sound = AudioSegment.from_mp3(mp3[itr1])
        sound.export(all_speakers[itr1]+'/'+'.wav', format="wav")
        
      wavs=glob.glob(all_speakers[itr1]+'/*.wav')
      
      spk=(all_speakers[itr1]).split("/")[-1]
      
      if not os.path.exists(directory):
          os.makedirs(directory)
      
      final_feat=np.empty([0, 39])
      
      for itr2 in range(0,len(wavs)):

          
          final_feat = eng.prosody(wavs[itr2])

          print(final_feat.shape)
      print(spk)    
      np.savetxt(feat_train_path+spk+"_all_features.txt", final_feat, delimiter=",")

      try:
          gmm = mixture.GaussianMixture(n_components=n_mixtures, covariance_type='diag' , max_iter = max_iterations ).fit(final_feat)
      except:
          print("ERROR : Error while training model for file "+spk)
          
      try:
          joblib.dump(gmm,trained_model_path+spk+'.pkl')
      except:
          print("ERROR : Error while saving model for "+spk)
          

  print("Training Completed")


Testing Function

In [ ]:
def testing(test_data_path,feat_test,trained_model_path):
    # train feature extraction
  all_speakers=glob.glob(test_data_path+'*')

  import os
  directory=feat_test
  if not os.path.exists(directory):
      os.makedirs(directory)

  speakers = { all_speakers[k]:k for k in range(len(all_speakers)) }

  num_test_cases={}
  tct={}
  for e in speakers:
      num_test_cases[e.replace(test_data_path,'')]=len(os.listdir(e))-1
      tct[e.replace(test_data_path,'')]=0

  print(num_test_cases)

  spk_names = { all_speakers[k].replace(test_data_path,''):k for k in range(len(all_speakers)) }

  total_speakers=len(num_test_cases)

  confusion_matrix = np.zeros((total_speakers,total_speakers))


  for itr1 in range(0,len(all_speakers)):
      
      wavs=glob.glob(all_speakers[itr1]+'/*.wav')
      
      spk=(all_speakers[itr1]).split("/")[-1]
      
      if not os.path.exists(directory):
          os.makedirs(directory)
      
      final_feat=np.empty([0, 39])
      output = {"Test_i":[],"Accent":[]}
      for itr2 in range(0,len(wavs)):
          #print(wavs[itr2])

          
          feat = eng.prosody(wavs[itr2])
          
          final_feat=np.concatenate((final_feat,feat),axis=0)
          
          #print(final_feat.shape)
          max_score=-np.inf
          max_spk_name=""
          
          for modelfile in sorted(glob.glob(trained_model_path+'*.pkl')):
              gmm = joblib.load(modelfile) 
              score=gmm.score(feat)
              #print score
              if score>max_score:
                  max_score,max_spk_name=score,modelfile.replace(trained_model_path,'').replace('.pkl','')

          print(spk+" -> "+max_spk_name+(" Y" if spk==max_spk_name  else " N"))
          output ["Test_i"].append(wavs[itr2])
          output ["Accent"].append(max_spk_name)
          confusion_matrix[ spk_names[spk] ][spk_names[max_spk_name]]+=1
          tct[spk]+=1

          
      #print(spk)
      json_object = json.dumps(output, indent = 4)
      with open("Haram.json", "w") as outfile:
            outfile.write(json_object)
            
      np.savetxt(feat_test+spk+"_all_features.txt", feat, delimiter=",")
  return tct,confusion_matrix,total_speakers
      

File Paths

In [ ]:
feat='/feat/'
feat_train='/train/'
feat_test='/test/'
trained_model='/train_models/'
train_data='/traindata/'
test_data='/testdata/'

# for removing existing feature folders, models created
if os.path.exists('/feat/'):
  !rm -rf '/feat/'
if os.path.exists('/train_models/'):
  !rm -rf '/train_models/'

Training

In [ ]:
training(train_data,feat_train,trained_model)

Training Completed


Testing

In [ ]:
tt,conf_mat,tot_spek=testing(test_data,feat_test,trained_model)

{}


Accuracy

In [ ]:
print(tt)
print("Confusion Matrix:\n",conf_mat)
print("Accuracy: ",(sum([ conf_mat[i][j] if i==j  else 0 for i in range(tot_spek) for j in range(tot_spek) ] )*100)/float(sum([i for i in tt.values()])))

{}
Confusion Matrix:
 []


ZeroDivisionError: float division by zero